<a href="https://colab.research.google.com/github/koyomin9zx/UITQA-Vietnamese-Question-Answering/blob/master/example_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/koyomin9zx/UITQA-Vietnamese-Question-Answering.git

Cloning into 'UITQA-Vietnamese-Question-Answering'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 88 (delta 40), reused 23 (delta 6), pack-reused 0
Unpacking objects: 100% (88/88), done.


In [0]:
!pip install pytorch-transformers
!pip install underthesea 
!pip install unidecode

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!ln -s /content/drive/'My Drive'/data

In [0]:
!mv /content/UITQA-Vietnamese-Question-Answering/combine.py /content

In [0]:
# importing libraries 
from underthesea import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
from scipy.spatial import distance 
from collections import defaultdict, OrderedDict 
from string import punctuation
import unidecode
import re
import pandas as pd 
import numpy as np 
import glob
from combine import QA
import time


In [7]:
start = time.time()
model=QA('/content/data/BERT_Squad_WIki_UIT_pretrain') #path to model
end = time.time()
print("time load model: "+str(round((end - start),2)))

time load model: 34.13


In [0]:
def load_data(path):
  data=[]
  all_files = glob.glob(path + "/*.txt")
  for file in all_files:
      passage=open(file, "r", encoding='utf-8').read()
      data.append(passage)
  return data

def vi_tokenizer(row):
    return word_tokenize(row, format="text")

def remove_stopwords(stopwords,text):
  sent = [s for s in text.split() if s not in stopwords ]
  sent = ' '.join(sent)
  return sent


def remove_punctuation(row):
  remove = punctuation
  remove = remove.replace("_", "")
  pattern = "[{}]".format(remove) # create the pattern
  re_space=re.compile('\s+')
  re_trailing=re.compile('^\s+|\s+?$')
  row=re.sub(pattern, " ", row) 
  row=re.sub(re_space,' ',row)
  row=re.sub(re_trailing,' ',row)
  row = row.strip()
  row =row.lower()
  return row

def standardize_data(df,stopwords):
    hl_cleansed=[]
    remove = punctuation
    remove = remove.replace("_", "")
    remove = remove.replace("(", "")
    remove = remove.replace(")", "")
    pattern = "[{}]".format(remove) # create the pattern
    re_space=re.compile('\s+')
    re_trailing=re.compile('^\s+|\s+?$')
    for row in df:
        row = vi_tokenizer(row)
        row=re.sub(pattern, " ", row) 
        row=re.sub(re_space,' ',row)
        row=re.sub(re_trailing,' ',row)
        row = row.strip()
        row = remove_stopwords(stopwords,row)
        #row = remove_accents(row)
        row = row.lower()
        hl_cleansed.append(row)
    return hl_cleansed


def sentences_tokenize(text):
    sents = sent_tokenize(text)
    sents = [word_tokenize(s,format = 'text') for s in sents]
    sents = [remove_punctuation(s) for s in sents]
    sents = [s.lower() for s in sents]
    #sents = [remove_stopwords(stopwords,s) for s in sents]
    return sents


## Converting 3D array of array into 1D array 
def arr_convert_1d(arr): 
    arr = np.array(arr) 
    arr = np.concatenate( arr, axis=0 ) 
    arr = np.concatenate( arr, axis=0 ) 
    return arr 
  
## Cosine Similarity 
def cosine(trans): 
    cos = [] 
    cos.append(cosine_similarity(trans[0], trans[1])) 
    return cos

def tfidf(str1, str2,tf_idf_vetor,stopwords):
    str1=standardize_data([str1],stopwords)
    str2=standardize_data([str2],stopwords)  
    corpus = [str1[0],str2[0]] 
    trans = tf_idf_vetor.transform(corpus)
    cos=cosine(trans) 
    return arr_convert_1d(cos)[0]

def relevant_ranking(query,data,vect,stopwords):
  query=standardize_data([query],stopwords)[0]
  print('Query: ',query,'\n')
  score=defaultdict()
  i=0
  for d in data:
    t=tfidf(query, d,vect,stopwords)
    if t!=0.0:
      score[t]=d
    i+=1
  return OrderedDict(sorted(score.items(),reverse=True))



In [122]:
data=load_data('/content/data/data_QA/data')
stopwords = set(open('/content/data/data_QA/stopwords/stopwords.txt').read().split(' ')[:-1])

data_standard=standardize_data(data,stopwords)
vect = TfidfVectorizer(min_df=1, max_df=0.8,max_features=5000,sublinear_tf=True) 
vect.fit(data_standard)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [134]:
q="năm 1912 Hồ Chí Minh làm gì?"

a=relevant_ranking(q,data_standard,vect,stopwords)


b=standardize_data([q],stopwords)[0]
i=0
for score,doc in a.items():
  answer = model.predict(doc,q)
  num_overlap=len(set(b.split()) & set(doc.split()))
  if answer['confidence']>0.1 and num_overlap >1:
    print('\nQuestion: ',q)
    print('\nOverlap key word: ',num_overlap)
    print('\nAnswer: ',answer['answer'])
    print('\nIR Sccore: ',score)
    print('\nBert Score: ',answer['confidence'])
    print('\nContent: ',doc,'...')
    print('\n==========================================\n\n\n')
    i+=1
  if i==5:
    break



Query:  năm 1912 hồ_chí_minh làm 


Question:  năm 1912 Hồ Chí Minh làm gì?

Overlap key word:  4

Answer:  người rời tổ_quốc đi sang phương tây tìm con đường giải_phóng dân_tộc

IR Sccore:  0.08085411191360731

Bert Score:  0.9568646332495551

Content:  hồ_chí_minh cuộc_đời chủ_tịch hồ_chí_minh cuộc_đời trong_sáng cao_đẹp người cộng_sản vĩ_đại anh_hùng dân_tộc kiệt_xuất chiến_sĩ quốc_tế lỗi_lạc đấu_tranh mệt_mỏi hiến_dâng đời mình tổ_quốc nhân_dân lý_tưởng cộng_sản chủ_nghĩa độc_lập tự_do dân_tộc hòa_bình công_lý thế_giới chủ_tịch hồ_chí_minh ( nhỏ tên nguyễn_sinh_cung đi học nguyễn_tất_thành trong năm hoạt đông cách_mạng lấy tên nguyễn_ái_quốc bí_danh bút_danh khác ) sinh ngày 19 tháng 5 năm 1890 ở kim_liêm huyện nam_đàn tình nghệ_an mất ngày 2 tháng 9 năm 1969 hà_nội người sinh trong gia_đình nhà_nho yêu nước lớn ở địa_phương truyền_thống yêu nước anh_dũng chống giặc ngoại_xâm sống trong hoàn_cảnh đất_nước chìm ách đô_hộ thực_dân pháp thời niên_thiếu thanh_niên người chứng_kiến nỗi 